[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Humboldt-WI/bads/blob/master/tutorials/9_nb_feature_engineering.ipynb) 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/code
!pip install --upgrade pip
!pip install -r requirements.txt

/content/drive/MyDrive/code
     |████████████████████████████████| 2.1 MB 13.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 63.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.2/136.2 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.0/837.0 KB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.9/793.9 KB 35.7 MB/s eta 0:00:00
     ━━━━

In [3]:
path_to_data_folder = '/content/drive/MyDrive/'

In [4]:
# Import standard Python libraries
import numpy as np
import pandas as pd
import os
import time

### Get PecanStreet Data:

Download the data from Dataport https://dataport.pecanstreet.org/academic (Account needed!)
data: New York 1-min (121 MB)
metadata: Metadata Report
Weatherdata is purchased from https://openweathermap.org/
Selection process New York: 
1. Check which houses have data on relevant appliances 
2. Choose houses with 100% data (2 houses)
3. Selecting ID: 3996

In [5]:
df = pd.read_csv(f'{path_to_data_folder}data/PecanSD/1minute_data_newyork.csv')
#metadata = pd.read_csv(f'{path_to_data_folder}data/PecanSD/metadata.csv')
df_weather = pd.read_csv(f'{path_to_data_folder}data/PecanSD/2019-soils-weather.csv')

df = df[['dataid', 'localminute' , 'clotheswasher1','dishwasher1','drye1', 'lights_plugs1', 'refrigerator1']]

In [6]:
df_3996 = df[df['dataid']==3996]
#df_3996.to_csv(f'{path_to_data_folder}data/PecanSD/1minute_data_newyork_3996.csv', index=False)

# further processing of house df_3996
#df_3996 = pd.read_csv(f'{path_to_data_folder}data/PecanSD/1minute_data_newyork_3996.csv')
df_3996 = df_3996[['localminute', 'clotheswasher1', 'dishwasher1', 'drye1', 'refrigerator1']]
df_3996.rename(columns={'localminute':'Time', 'clotheswasher1':'Washing Machine', 'dishwasher1': 'Dishwasher', 'drye1':'Dryer', 'refrigerator1':'Fridge'}, inplace=True)

In [7]:
df_3996['Time'] = pd.to_datetime(df_3996['Time']).dt.tz_localize(None)
df_3996.set_index('Time', inplace=True)
for col in list(df_3996.columns):
  df_3996[col] = pd.to_numeric(df_3996[col])
  df_3996[col] = df_3996[col]*1000
df_3996 = df_3996.resample('1H').mean()

In [8]:
df_weather.drop_duplicates(inplace=True)
df_weather = df_weather[['localhour', 'temperature', 'humidity', 'wind_speed', 'location']]
df_weather = df_weather[df_weather['location']=='New York2']
df_weather['localhour'] = pd.to_datetime(df_weather['localhour']).dt.tz_localize(None)

mask1 = (df_weather['localhour'] >= '2019-05-01 00:00:00') & (df_weather['localhour'] <= '2019-10-31 23:00:00')
df_weather = df_weather.loc[mask1]

In [9]:
idx = pd.date_range(start = min(df_weather['localhour']), end = max(df_weather['localhour']),freq='H')
df_weather['localhour'] = pd.to_datetime(df_weather['localhour'])
df_weather.set_index('localhour', inplace=True)
df_weather = df_weather.groupby(df_weather.index).mean()
df_weather = df_weather.reindex(idx)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
pecan = pd.concat([df_3996, df_weather], axis=1, join="inner")
pecan = pecan.reset_index()   
pecan = pecan[['index', 'Fridge', 'Washing Machine', 'Dishwasher', 'Dryer', 'temperature', 	'humidity', 	'wind_speed']]
pecan = pecan.rename(columns={'index':'Time', 'temperature': 'Temperature', 'humidity':'Relative Humidity', 'wind_speed': 'Wind Speed'})

In [11]:
pecan_train = pecan.iloc[:-864]
pecan_test = pecan.iloc[-864:]
pecan_train.to_csv(f'{path_to_data_folder}data/PecanSD/3996_train.csv', index=False)
pecan_test.to_csv(f'{path_to_data_folder}data/PecanSD/3996_test.csv', index=False)

### Get GreenD Data:


Download data from https://docs.google.com/forms/d/e/1FAIpQLSf3Tbr7IDoSORNFw7dAGD2PB6kSO98RRiVpmKOWOZ52ULAMzA/viewform (after filling out the form)

Weatherdata is purchased from https://openweathermap.org/

In [12]:
appl_lst = ['Television','Fridge', 'Washing Machine', 'Dishwasher'] 
dt_lst = []

with os.scandir(f'{path_to_data_folder}data/GreenD/building0') as it:
    for entry in it:
        if not (entry.name.startswith('date.txt') or entry.name.startswith('dataset_2013-12-07') or entry.name.startswith('.')) and entry.is_file():
            print(entry.name)
            data = pd.read_csv(f'{path_to_data_folder}data/GreenD/building0/{entry.name}')
            data['timestamp'] = data['timestamp'].astype(str)
            data['timestamp'] = data['timestamp'].str.replace(r'timestamp', '0')
            data['timestamp'] = data['timestamp'].astype(float)
            data['timestamp'] = data['timestamp'].apply(lambda x: time.strftime('%A, %Y-%m-%d %H:%M:%S', time.localtime(x))) 
            data['timestamp'] = pd.to_datetime(data['timestamp'], format='%A, %Y-%m-%d %H:%M:%S', errors='ignore')
            data = data[data['timestamp']!='1970-01-01 00:00:00'].copy()
            data['timestamp'] = pd.to_datetime(data['timestamp'])
            data.set_index('timestamp', inplace=True)
            # building0
            data.rename(columns={'000D6F0002907BA2':'Washing Machine','000D6F0002907BF5':'Fridge', '000D6F0002907C89':'Dishwasher', '000D6F0002908162' :'Television'}, inplace=True)        
            data = data[appl_lst]
            for col in appl_lst:
              data[col] = pd.to_numeric(data[col])
            data = data.resample('1H').mean()
            dt_lst.append(data)

buildin0_df = pd.concat(dt_lst)
buildin0_df = buildin0_df.sort_index()
buildin0_df.to_csv(f'{path_to_data_folder}data/GreenD/buildin0_df.csv')    

dataset_2014-08-28.csv
dataset_2014-09-02.csv
dataset_2014-04-28.csv
dataset_2014-06-30.csv
dataset_2014-07-21.csv
dataset_2014-04-29.csv
dataset_2014-03-16.csv
dataset_2014-02-15.csv
dataset_2014-03-06.csv
dataset_2014-07-11.csv
dataset_2014-08-15.csv
dataset_2013-12-11.csv
dataset_2014-07-10.csv
dataset_2014-07-31.csv
dataset_2014-04-19.csv
dataset_2014-06-20.csv
dataset_2014-09-23.csv
dataset_2014-03-24.csv
dataset_2014-04-14.csv
dataset_2014-10-05.csv
dataset_2014-04-18.csv
dataset_2014-03-17.csv
dataset_2014-04-16.csv
dataset_2014-03-19.csv
dataset_2014-03-26.csv
dataset_2014-01-14.csv
dataset_2014-03-14.csv
dataset_2014-01-11.csv
dataset_2014-02-16.csv
dataset_2014-03-09.csv
dataset_2014-07-24.csv
dataset_2014-07-03.csv
dataset_2014-03-25.csv
dataset_2014-04-25.csv
dataset_2014-08-04.csv
dataset_2014-04-22.csv
dataset_2014-08-11.csv
dataset_2013-12-19.csv
dataset_2014-07-28.csv
dataset_2014-08-27.csv
dataset_2013-12-28.csv
dataset_2014-06-29.csv
dataset_2014-09-17.csv
dataset_201

In [13]:
buildin0_df.head(2)

,Television,Fridge,Washing Machine,Dishwasher
timestamp,,,,
2013-12-07 23:00:00,154.918894,9.282116,0.0,0.215133
2013-12-08 00:00:00,61.082069,35.234375,0.0,0.182931


In [14]:
buildin0_df = pd.read_csv(f'{path_to_data_folder}data/GreenD/buildin0_df.csv')
buildin0_df['timestamp'] = pd.to_datetime(buildin0_df['timestamp'])

idx = pd.date_range(start = min(buildin0_df['timestamp']), end = max(buildin0_df['timestamp']),freq='H')
buildin0_df['timestamp'] = pd.to_datetime(buildin0_df['timestamp'])
buildin0_df.set_index('timestamp', inplace=True)
buildin0_df = buildin0_df.groupby(buildin0_df.index).mean()
buildin0_df = buildin0_df.reindex(idx)
mask1 = (buildin0_df.index > '2013-12-07 22:00:00') & (buildin0_df.index <= '2014-04-29 19:00:00')
mask2 = (buildin0_df.index > '2014-06-17 10:00:00') & (buildin0_df.index <= '2014-10-13 12:00:00')
buildin0_df1 = buildin0_df.loc[mask1]
buildin0_df2 = buildin0_df.loc[mask2]
frames2 = [buildin0_df1, buildin0_df2]
buildin0_df = pd.concat(frames2)

In [15]:
weather_build0 = pd.read_csv(f'{path_to_data_folder}data/GreenD/weather_greend.csv')
weather_build0.drop(columns=['dt_iso', 'timezone', 	'city_name', 	'lat', 	'lon', 'wind_deg', 	'wind_gust'], inplace=True)
weather_build0['dt'] = weather_build0['dt'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))) 
weather_build0['dt'] = pd.to_datetime(weather_build0['dt'])
weather_build0.rename(columns={'dt':'timestamp'}, inplace=True)

mask1 = (weather_build0['timestamp'] > '2013-12-07 22:00:00') & (weather_build0['timestamp'] <= '2014-04-29 19:00:00')
mask2 = (weather_build0['timestamp'] > '2014-06-17 10:00:00') & (weather_build0['timestamp'] <= '2014-10-13 12:00:00')
weather_build01 = weather_build0.loc[mask1]
weather_build02 = weather_build0.loc[mask2]
frames = [weather_build01, weather_build02]
weather_build0 = pd.concat(frames)
weather_build0.set_index('timestamp', inplace=True)

In [16]:
greend0 = pd.concat([buildin0_df, weather_build0], axis=1, join="inner")

greend0 = greend0.reset_index()   
greend0 = greend0[['index', 'Fridge', 'Washing Machine', 'Dishwasher', 'Television', 'temp', 	'humidity', 	'wind_speed']]
greend0 = greend0.rename(columns={'index':'Time', 'temp': 'Temperature', 'humidity':'Relative Humidity', 'wind_speed': 'Wind Speed'})

In [17]:
buildin0_df_test = greend0.iloc[-600:]
buildin0_df_train = greend0.iloc[:-600]
buildin0_df_train.to_csv(f'{path_to_data_folder}data/GreenD/buildin0_df_train.csv', index=False)
buildin0_df_test.to_csv(f'{path_to_data_folder}data/GreenD/buildin0_df_test.csv', index=False)

### Get REFIT


To get the REFIT data visit https://www.refitsmarthomes.org/datasets/ and click the provided data links
data download: REFIT Electrical Load Measurements (Cleaned) (University of Strathclyde) 
metadata and weatherdata download: REFIT Smart Home dataset (Loughborough University)

In [18]:
data_ref = pd.read_csv(f'{path_to_data_folder}data/REFIT/CLEAN_House1.csv')
data_ref = data_ref.drop(columns=['Aggregate','Issues', 'Unix','Appliance2','Appliance3','Appliance7','Appliance9' ])
data_ref.rename(columns={'Appliance1':'Fridge', 'Appliance4':'Dryer', 'Appliance5': 'Washing Machine', 'Appliance6': 'Dishwasher', 'Appliance8': 'Television'}, inplace=True)
data_ref.head(2)

,Time,Fridge,Dryer,Washing Machine,Dishwasher,Television
0,2013-10-09 13:06:17,74,0,0,0,0
1,2013-10-09 13:06:31,75,0,0,0,0


In [19]:
data_ref['Time'] = pd.to_datetime(data_ref['Time'])
data_ref.set_index('Time', inplace=True)
data_ref = data_ref.resample('1H').mean()


In [20]:
import xml.etree.ElementTree as ET
tree = ET.parse(f'{path_to_data_folder}data/REFIT/REFIT_BUILDING_SURVEY.xml')
root = tree.getroot()
print(root[0][0][2][0].attrib) #TimeSeriesVariable1573 / 'Air temperature'
#TimeSeriesVariable1574 / 'Relative humidity'
# 'TimeSeriesVariable1575' / 'Wind speed'


{'id': 'TimeSeriesVariable1575', 'startDateTime': '2013-11-04T09:15:00Z', 'endDateTime': '2015-06-01T14:45:00Z', 'variableType': 'Wind speed', 'units': 'm/s', 'intervalType': 'FixedInterval', 'intervalUnit': 'Minute', 'intervalLength': '15', 'hasMissingData': 'No', 'repeatsOmitted': 'No', 'hasDuplicateTimestamps': 'No'}


In [21]:
weather = pd.read_csv(f'{path_to_data_folder}data/REFIT/REFIT_TIME_SERIES_VALUES.csv')

temp_df = weather[weather['TimeSeriesVariable/@id']=='TimeSeriesVariable1573']
temp_df.drop(columns=['TimeSeriesVariable/@id'], inplace=True)
temp_df.rename(columns={'dateTime':'Time', 'data': 'Temperature'}, inplace=True)
temp_df['Time'] = pd.to_datetime(temp_df['Time'], format="%Y-%m-%dT%H:%M:%S%fZ") 
temp_df['Time'] = pd.to_datetime(temp_df['Time'])
temp_df.set_index('Time', inplace=True)
temp_df = temp_df.resample('1H').mean()


humid_df = weather[weather['TimeSeriesVariable/@id']=='TimeSeriesVariable1574']
humid_df.drop(columns=['TimeSeriesVariable/@id'], inplace=True)
humid_df.rename(columns={'dateTime':'Time', 'data': 'Relative Humidity'}, inplace=True)
humid_df['Time'] = pd.to_datetime(humid_df['Time'], format="%Y-%m-%dT%H:%M:%S%fZ") 
humid_df['Time'] = pd.to_datetime(humid_df['Time'])
humid_df.set_index('Time', inplace=True)
humid_df = humid_df.resample('1H').mean()

winds_df = weather[weather['TimeSeriesVariable/@id']=='TimeSeriesVariable1575']
winds_df.drop(columns=['TimeSeriesVariable/@id'], inplace=True)
winds_df.rename(columns={'dateTime':'Time', 'data': 'Wind Speed'}, inplace=True)
winds_df['Time'] = pd.to_datetime(winds_df['Time'], format="%Y-%m-%dT%H:%M:%S%fZ") 
winds_df['Time'] = pd.to_datetime(winds_df['Time'])
winds_df.set_index('Time', inplace=True)
winds_df = winds_df.resample('1H').mean()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/py

In [22]:
weather_df = pd.concat([temp_df, humid_df, winds_df], axis=1, join="inner")
refit_combined = pd.concat([data_ref, weather_df], axis=1, join="inner")
refit_combined.dropna(inplace=True)
refit_train = refit_combined[:9277]
refit_test = refit_combined[9277:]
refit_train.to_csv(f'{path_to_data_folder}data/REFIT/refit_h1_full.csv')
refit_test.to_csv(f'{path_to_data_folder}data/REFIT/refit_test.csv')

### AMPds

To download the AMPds data visit http://ampds.org/
data download: Electricity_P.csv
weather data: https://dataverse.harvard.edu/file.xhtml?persistentId=doi:10.7910/DVN/FIE0S4/XH3RRJ&version=3.1


In [23]:
weather = pd.read_csv(f'{path_to_data_folder}data/AMPds/Climate_HourlyWeather.csv')
weather = weather[['Date/Time', 'Temp (C)', 'Rel Hum (%)','Wind Spd (km/h)' ]]
weather.rename(columns={'Date/Time': 'Time','Temp (C)': 'Temperature', 'Rel Hum (%)': 'Relative Humidity','Wind Spd (km/h)': 'Wind Speed' }, inplace=True)
weather['Time'] = pd.to_datetime(weather['Time'])
weather.set_index('Time', inplace=True)

In [24]:
data_ampds = pd.read_csv(f'{path_to_data_folder}data/AMPds/Electricity_P.csv') 
data_ampds = data_ampds[['UNIX_TS', 'FGE', 'DWE', 'CWE', 'CDE', 'TVE']]
data_ampds.rename(columns={'UNIX_TS': 'Time', 'FGE':'Fridge', 'DWE':'Dishwasher', 'CWE': 'Washing Machine', 'CDE': 'Dryer', 'TVE':'Television'}, inplace=True)
data_ampds['Time'] = pd.to_datetime(data_ampds['Time'], unit='s')
data_ampds.set_index('Time', inplace=True)
data_ampds = data_ampds.resample('1H').mean()
data_ampds = data_ampds.set_index(weather.index)


In [25]:
ampds_combined = pd.concat([weather, data_ampds], axis=1)
ampds_combined.dropna(inplace=True)
ampds_train = ampds_combined[:14596]
ampds_test = ampds_combined[14596:]
ampds_train.to_csv(f'{path_to_data_folder}data/AMPds/ampds_full.csv')
ampds_test.to_csv(f'{path_to_data_folder}data/AMPds/ampds_test.csv')